In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
import os
from sklearn import metrics


In [ ]:
#데이터 불러오기
data = pd.read_csv(r'2021_2022_분석_데이터_범주형.csv')

# 전처리

In [ ]:
data.loc[data['PM10'] <= 30.0, 'PM10_category'] = 0
data.loc[(data['PM10'] > 30.0) & (data['PM10'] <= 80.0), 'PM10_category'] = 1
data.loc[(data['PM10'] > 80.0) & (data['PM10'] <= 150.0), 'PM10_category'] = 2
data.loc[(data['PM10'] > 150.0), 'PM10_category'] = 3

data = data.set_index('일시')
data = data.drop(['PM2.5','PM10'], axis = 1)
data = data.apply(pd.to_numeric, errors='coerce')

# data.head()

In [ ]:
#악성 양성 차이
data['PM10_category'].value_counts()

In [ ]:
#Visualization
sns.countplot(x="PM10_category", data=data)
plt.show()

In [ ]:
data1 = data

In [ ]:
#X,y 정하기
y = data1['PM10_category']
X = data1.drop('PM10_category', axis=1)

#train, test 분류. test_size=0.2
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2, shuffle = False)

#표준화
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
sc.fit(X_train)
X_train_std=sc.transform(X_train)
X_test_std=sc.transform(X_test)

# 로지스틱 회귀

In [ ]:
#그리드 써치
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
param_grid = {'C':[1.0, 10, 50, 100], 'max_iter':[100,200,300,400]}
grid = GridSearchCV(lr, param_grid=param_grid, cv=10, refit=True)
grid.fit(X_train_std, y_train)


test_score = grid.score(X_test_std, y_test)
print("Test set 점수: {}".format( test_score ))
train_score = grid.score(X_train_std, y_train)
print("Train set 점수: {}".format(train_score))
print("최적 매개변수: {}".format(grid.best_params_))
print("최고 교차 검증 점수: {}".format(grid.best_score_))

In [ ]:
Logit=LogisticRegression(C=10,max_iter=300) # 
Logit.fit(X_train_std,y_train)
y_train_pred=Logit.predict(X_train_std)
y_test_pred=Logit.predict(X_test_std)

from sklearn import metrics
print('Train : ', round(metrics.accuracy_score(y_train,y_train_pred),3))
print('Test : ', round(metrics.accuracy_score(y_test,y_test_pred),3))

In [ ]:
print('좋음으로 잘 분류된 확률>> ', round(y_train.value_counts()[0]/pd.Series(y_train_pred).value_counts()[0],3), '\n',
      '보통으로 잘 분류된 확률>> ', round(y_train.value_counts()[1]/pd.Series(y_train_pred).value_counts()[1],3), '\n', 
      '나쁨으로 잘 분류된 확률>> ', round(y_train.value_counts()[2]/pd.Series(y_train_pred).value_counts()[2],3))

In [ ]:
#confusion matrix
print(metrics.confusion_matrix(y_test,y_test_pred))

# KNN

In [ ]:
#그리드써치
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
param_grid = {'n_neighbors':[4,5,6,7]}
grid = GridSearchCV(knn, param_grid=param_grid, cv=10, refit=True)
grid.fit(X_train, y_train)


test_score = grid.score(X_test, y_test)
print("시험 set 점수: {}".format( test_score ))
train_score = grid.score(X_train, y_train)
print("학습 set 점수: {}".format(train_score))
print("최적 매개변수: {}".format(grid.best_params_))
print("최고 교차 검증 점수: {}".format(grid.best_score_))

In [ ]:
knn=KNeighborsClassifier(n_neighbors=7,p=2)
knn.fit(X_train,y_train)
y_train_pred=knn.predict(X_train)
y_test_pred=knn.predict(X_test)

#정확도 출력
from sklearn import metrics

print('Train : ', round(metrics.accuracy_score(y_train,y_train_pred),3))
print('Test : ', round(metrics.accuracy_score(y_test,y_test_pred),3))

In [ ]:
#혼동행렬
print(metrics.confusion_matrix(y_test,y_test_pred))

# decisionTree

In [ ]:
#그리드 써치
from sklearn.model_selection import GridSearchCV
from sklearn import tree
dtc = tree.DecisionTreeClassifier()

In [ ]:
param_grid = {'criterion':['gini','entropy'], 'max_depth':[None,2,3,4,5,6], 'min_samples_split':[2,3,4,5,6], 
              'min_samples_leaf':[1,2,3]}
grid = GridSearchCV(dtc, param_grid=param_grid, cv=10, refit=True)
grid.fit(X_train_std, y_train)

test_score = grid.score(X_test_std, y_test)
print("시험 set 점수: {}".format( test_score ))
train_score = grid.score(X_train_std, y_train)
print("학습 set 점수: {}".format(train_score))
print("최적 매개변수: {}".format(grid.best_params_))
print("최고 교차 검증 점수: {}".format(grid.best_score_))

In [ ]:
from sklearn import tree
dtc=tree.DecisionTreeClassifier(criterion='entropy',max_depth=3,min_samples_leaf=3,min_samples_split=4,random_state=1)
result=dtc.fit(X_train_std,y_train)
y_train_pred=dtc.predict(X_train_std)
y_test_pred=dtc.predict(X_test_std)

#정확도 출력
print('Train : ', round(metrics.accuracy_score(y_train,y_train_pred),3))
print('Test : ', round(metrics.accuracy_score(y_test,y_test_pred),3))

In [ ]:
#confusion matrix
print(metrics.confusion_matrix(y_test,y_test_pred))

In [ ]:
from pydotplus import graph_from_dot_data
from sklearn.tree import export_graphviz
dot_data=export_graphviz(dtc, filled=True, rounded=True, class_names=['B','M'],
                         feature_names=['radius_mean','texture_mean','smoothness_mean','compactness_mean','concavity_mean',
                                        'symmetry_mean','fractal_dimension_mean','radius_se','texture_se','smoothness_se',
                                        'compactness_se','concavity_se','concave points_se','symmetry_se','fractal_dimension_se',
                                       'smoothness_worst','compactness_worst','concavity_worst','symmetry_worst','fractal_dimension_worst'],
                         out_file=None)
graph=graph_from_dot_data(dot_data)
graph.write_png('tree.png')

In [ ]:
from IPython.display import Image
Image(graph.create_png())

# SVM 

In [ ]:
y = data1['PM10_category']
X = data1.drop('PM10_category', axis=1)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2) # 주성분을 몇개로 할지 결정
printcipalComponents = pca.fit_transform(X)
principalDf = pd.DataFrame(data=printcipalComponents, columns = ['principal component1', 'principal component2'])
# 주성분으로 이루어진 데이터 프레임 구성
print(pca.explained_variance_ratio_, sum(pca.explained_variance_ratio_))

In [ ]:
#SVM
from sklearn.model_selection import train_test_split

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
#train, test 분류. test_size=0.2
X_train, X_test, y_train, y_test=train_test_split(printcipalComponents,y,test_size=0.2, shuffle = False)

In [ ]:
#표준화
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
sc.fit(X_train)
X_train_std=sc.transform(X_train)
X_test_std=sc.transform(X_test)

In [ ]:
#그리드 써치
#매개변수
param_grid = {'C': [0.01, 0.1, 1, 10],
             'gamma': [0.01, 0.1, 1, 10] }

svm = SVC()
grid_ksvm = GridSearchCV(SVC(kernel='rbf'), param_grid, cv=10, return_train_score=True)
grid_ksvm.fit(X_train_std, y_train)

In [ ]:
# 테스트 세트 점수
test_score = grid_ksvm.score(X_test_std, y_test)
print("시험 set 점수: {}".format( test_score ))
train_score = grid_ksvm.score(X_train_std, y_train)
print("학습 set 점수: {}".format(train_score))
print("최적 매개변수: {}".format(grid_ksvm.best_params_))
print("최고 교차 검증 점수: {}".format(grid_ksvm.best_score_))

In [ ]:
a = [0.1, 1, 10]
for i in a:
    ksvm=SVC(kernel='poly', C=i, gamma=5)
    ksvm.fit(X_train_std,y_train)
    y_train_pred=ksvm.predict(X_train_std)
    y_test_pred=ksvm.predict(X_test_std)
    #정확도
    from sklearn import metrics
    print('\n\n', i, 5, '')
    print('Train : ', round(metrics.accuracy_score(y_train,y_train_pred),3))
    print('Test : ', round(metrics.accuracy_score(y_test,y_test_pred),3))

In [ ]:
print(y_train.value_counts(),'\n\n', pd.Series(y_train_pred).value_counts())

In [ ]:
#confusion matrix
print(metrics.confusion_matrix(y_test,y_test_pred))

# 앙상블

In [ ]:
#train, test 분류. test_size=0.2
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2, shuffle = False)

# Random Forest

In [ ]:
#grid search
from sklearn.ensemble import RandomForestClassifier
# param_grid = {'max_depth': [1,2,3,4,5,6],
#              'min_samples_leaf': [2,3,4,5],'min_samples_split':[2,3,4,5] }
rf = RandomForestClassifier()
# grid_rf = GridSearchCV(rf, param_grid, cv=10)
# grid_rf.fit(X_train, y_train)

In [ ]:
print("최적 매개변수: {}".format(grid_rf.best_params_))

In [ ]:
aa = i, j, k;aa

In [ ]:
aa = []
bb = []
cc = []
df_traintest = ({'index':[]}, {'Train':[]}, {'Test':[]})
for l in [100,150,200, 250]:
    for i in [6, 7, 8, 9, 10]:
        for j in [2,3,4,5]:
            for k in [2,3,4,5]:
                rf = RandomForestClassifier()
                rf = RandomForestClassifier(n_estimators = l, max_depth=i,min_samples_leaf=j,min_samples_split=k)
                rf.fit(X_train_std,y_train)
                y_train_pred=rf.predict(X_train_std)
                y_test_pred=rf.predict(X_test_std)
                from sklearn import metrics
                print(l, i, j, k)
                aa = list[l, i, j, k]
                print(metrics.accuracy_score(y_train,y_train_pred)) #train data에 대한 accuracy
                bb = metrics.accuracy_score(y_train,y_train_pred)
                print(metrics.accuracy_score(y_test,y_test_pred))  #test data에 대한 accuracy
                cc = metrics.accuracy_score(y_test,y_test_pred)
                df_traintest.append(aa, bb, cc)
print(df_traintest)

In [ ]:
aa = []
bb = []
cc = []
df_traintest = {'index':[], 'Train':[], 'Test':[]}

for i in [6, 7, 8, 9, 10]:
    for j in [2, 3, 4, 5]:
        for k in [2, 3, 4, 5]:
            rf = RandomForestClassifier(n_estimators=200, max_depth=i, min_samples_leaf=j, min_samples_split=k)
            rf.fit(X_train_std, y_train)
            y_train_pred = rf.predict(X_train_std)
            y_test_pred = rf.predict(X_test_std)
            from sklearn import metrics
            print(200, i, j, k)
            aa.append('200, {0}, {1}, {2}'.format(i, j, k))
            print(metrics.accuracy_score(y_train, y_train_pred))  # train data에 대한 accuracy
            bb.append(metrics.accuracy_score(y_train, y_train_pred))
            print(metrics.accuracy_score(y_test, y_test_pred))  # test data에 대한 accuracy
            cc.append(metrics.accuracy_score(y_test, y_test_pred))

In [ ]:
df_traintest['index'] = aa
df_traintest['Train'] = bb
df_traintest['Test'] = cc
df_traintest = df_traintest.set_index('index')

aa_randomforest = aa
bb_randomforest = bb
cc_randomforest = cc
df_traintest_randomforest = df_traintest

df_traintest

In [ ]:
plt.rc('font', family='Malgun Gothic')

df1 = pd.DataFrame(df_traintest['Train'])
df2 = pd.DataFrame(df_traintest['Test'])
df2 = df2.set_index(df1.index)

result = pd.concat([df1, df2], axis=1)
result.columns = ['Train', 'Test']

result['Train'].plot(figsize=(40, 16), alpha=1.0)
result['Test'].plot(figsize=(40, 16), alpha=1.0)
plt.legend(fontsize=30)
plt.title("RandomForest", size = 30)
plt.xlabel("\n하이퍼파라미터", size = 25)
plt.ylabel("정확도", size = 25)
plt.tick_params(axis='x', labelsize=17)
plt.tick_params(axis='y', labelsize=17)
plt.show()

# Adaboost

In [ ]:
#그리드써치
from sklearn.ensemble import AdaBoostClassifier

abc=AdaBoostClassifier()
# param_grid={'n_estimators':[50,80,100,200],'learning_rate':[0.01,0.1,1.0]}

# grid_abc = GridSearchCV(abc, param_grid, cv=10)

# grid_abc.fit(X_train, y_train)
# print("최적 매개변수: {}".format(grid_abc.best_params_))

In [ ]:
aa = []
bb = []
cc = []
df_traintest = {'index':[], 'Train':[], 'Test':[]}

for i in [5, 6, 7, 8, 9]:
    for j in [50, 100, 150, 200]:
        for k in [0.01, 0.1, 1.0]:
            from sklearn.tree import DecisionTreeClassifier
            base_model = DecisionTreeClassifier(max_depth = i)
            abc = AdaBoostClassifier(base_estimator = base_model,n_estimators = j,learning_rate = k)
            abc.fit(X_train_std, y_train)
            
            y_train_pred=abc.predict(X_train_std)
            y_test_pred=abc.predict(X_test_std)
            print(i, j, k)
            aa.append('{0}, {1}, {2}'.format(i, j, k))

            print(metrics.accuracy_score(y_train,y_train_pred)) #train data에 대한 accuracy
            bb.append(metrics.accuracy_score(y_train, y_train_pred))

            print(metrics.accuracy_score(y_test,y_test_pred))  #test data에 대한 accuracy
            cc.append(metrics.accuracy_score(y_test, y_test_pred))


In [ ]:
df_traintest = pd.DataFrame(df_traintest)
df_traintest['index'] = aa
df_traintest['Train'] = bb
df_traintest['Test'] = cc
df_traintest = df_traintest.set_index('index')


aa_Adaboost = aa
bb_Adaboost = bb
cc_Adaboost = cc
df_traintest_Adaboost = df_traintest

df_traintest_Adaboost


In [ ]:
plt.rc('font', family='Malgun Gothic')

df1 = pd.DataFrame(df_traintest['Train'])
df2 = pd.DataFrame(df_traintest['Test'])
df2 = df2.set_index(df1.index)

result = pd.concat([df1, df2], axis=1)
result.columns = ['Train', 'Test']

result['Train'].plot(figsize=(40, 16), alpha=1.0)
result['Test'].plot(figsize=(40, 16), alpha=1.0)
plt.legend(fontsize=30)
plt.title("AdaBoost", size = 30)
plt.xlabel("\n하이퍼파라미터", size = 25)
plt.ylabel("정확도", size = 25)
plt.tick_params(axis='x', labelsize=17)
plt.tick_params(axis='y', labelsize=17)
plt.show()

# Gradient Boost

In [ ]:
#grid search
from sklearn.ensemble import GradientBoostingClassifier

gbc=GradientBoostingClassifier()
# param_grid={'n_estimators':[50,80,100,200],'learning_rate':[0.001, 0.01,0.1,1.0],'max_depth':[2,3,4,5,6,7]}

# grid_gbc = GridSearchCV(gbc, param_grid, cv=10)

# grid_gbc.fit(X_train, y_train)
# print("최적 매개변수: {}".format(grid_gbc.best_params_))

In [ ]:
aa = []
bb = []
cc = []
df_traintest = {'index':[], 'Train':[], 'Test':[]}

for i in [50, 100, 150, 200]:
    for j in [0.001, 0.01, 0.1, 1.0]:
        for k in [5,6,7,8,9]:
            gbc = GradientBoostingClassifier()
            gbc = GradientBoostingClassifier(n_estimators = i,learning_rate = j,max_depth=k)
            gbc.fit(X_train_std, y_train)
            
            y_train_pred=gbc.predict(X_train_std)
            y_test_pred=gbc.predict(X_test_std)
            print(i, j, k)
            aa.append('{0}, {1}, {2}'.format(i, j, k))

            print(metrics.accuracy_score(y_train,y_train_pred)) #train data에 대한 accuracy
            bb.append(metrics.accuracy_score(y_train, y_train_pred))

            print(metrics.accuracy_score(y_test,y_test_pred))  #test data에 대한 accuracy
            cc.append(metrics.accuracy_score(y_test, y_test_pred))


In [ ]:
df_traintest = pd.DataFrame(df_traintest)
df_traintest['index'] = aa
df_traintest['Train'] = bb
df_traintest['Test'] = cc
df_traintest = df_traintest.set_index('index')


aa_GradientBoost= aa
bb_GradientBoost = bb
cc_GradientBoost = cc
df_traintest_GradientBoost = df_traintest

df_traintest_GradientBoost

In [ ]:
plt.rc('font', family='Malgun Gothic')

df1 = pd.DataFrame(df_traintest['Train'])
df2 = pd.DataFrame(df_traintest['Test'])
df2 = df2.set_index(df1.index)

result = pd.concat([df1, df2], axis=1)
result.columns = ['Train', 'Test']

result['Train'].plot(figsize=(40, 16), alpha=1.0)
result['Test'].plot(figsize=(40, 16), alpha=1.0)
plt.legend(fontsize=30)
plt.title("GradientBoost", size = 30)
plt.xlabel("\n하이퍼파라미터", size = 25)
plt.ylabel("정확도", size = 25)
plt.tick_params(axis='x', labelsize=17)
plt.tick_params(axis='y', labelsize=17)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

feature_imp = pd.DataFrame(sorted(zip(gbc.feature_importances_,X.columns)), columns=['Value','Feature'])

plt.figure(figsize=(20, 12))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False)[:15])
# plt.title('LightGBM Feature importances')
plt.tight_layout()
plt.show()

In [ ]:
print(metrics.confusion_matrix(y_test,y_test_pred))

# XGBoost

In [ ]:
from sklearn.preprocessing import LabelEncoder

le=LabelEncoder()
le.fit(y_train)
y_train=le.transform(y_train)
le.fit(y_test)
y_test=le.transform(y_test)

In [ ]:
# grid search
import xgboost as xgb

# param_grid={'n_estimators':[50,80,100,200],'learning_rate':[0.01,0.1,1.0],'max_depth':[2,3,4,5,6,7]}

# grid_xgbc = GridSearchCV(xgbc, param_grid, cv=10)

# grid_xgbc.fit(X_train, y_train)
# print("최적 매개변수: {}".format(grid_xgbc.best_params_))

In [ ]:
aa = []
bb = []
cc = []
df_traintest = {'index':[], 'Train':[], 'Test':[]}

for i in [50, 100, 150, 200]:
    for j in [0.001, 0.01, 0.1, 1.0]:
        for k in [5,6,7,8,9]:
            xgbc=xgb.XGBClassifier()
            xgbc =  xgb.XGBClassifier(n_estimators = i,learning_rate = j,max_depth=k)
            xgbc.fit(X_train_std, y_train)
            
            y_train_pred=xgbc.predict(X_train_std)
            y_test_pred=xgbc.predict(X_test_std)
            print(i, j, k)
            aa.append('{0}, {1}, {2}'.format(i, j, k))

            print(metrics.accuracy_score(y_train,y_train_pred)) #train data에 대한 accuracy
            bb.append(metrics.accuracy_score(y_train, y_train_pred))

            print(metrics.accuracy_score(y_test,y_test_pred))  #test data에 대한 accuracy
            cc.append(metrics.accuracy_score(y_test, y_test_pred))

In [ ]:
df_traintest = pd.DataFrame(df_traintest)
df_traintest['index'] = aa
df_traintest['Train'] = bb
df_traintest['Test'] = cc
df_traintest = df_traintest.set_index('index')


aa_xgboost= aa
bb_xgboost = bb
cc_xgboost = cc
df_traintest_xgboost = df_traintest

df_traintest_xgboost

In [ ]:
plt.rc('font', family='Malgun Gothic')

df1 = pd.DataFrame(df_traintest['Train'])
df2 = pd.DataFrame(df_traintest['Test'])
df2 = df2.set_index(df1.index)

result = pd.concat([df1, df2], axis=1)
result.columns = ['Train', 'Test']

result['Train'].plot(figsize=(40, 16), alpha=1.0)
result['Test'].plot(figsize=(40, 16), alpha=1.0)
plt.legend(fontsize=30)
plt.title("GradientBoost", size = 30)
plt.xlabel("\n하이퍼파라미터", size = 25)
plt.ylabel("정확도", size = 25)
plt.tick_params(axis='x', labelsize=17)
plt.tick_params(axis='y', labelsize=17)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

feature_imp = pd.DataFrame(sorted(zip(xgbc.feature_importances_,X.columns)), columns=['Value','Feature'])

plt.figure(figsize=(20, 12))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False)[:15])
# plt.title('LightGBM Feature importances')
plt.tight_layout()
plt.show()